In [ ]:
!pip install --upgrade ultralytics -q
!pip install roboflow -q

In [ ]:
#Importing Libraries
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from roboflow import Roboflow
from IPython.display import Image, clear_output
from ultralytics import YOLO
import os
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Object Segmentation With Water**

In [ ]:
# Initialize Roboflow and download the dataset
rf = Roboflow(api_key="cY2YZlXNZGwP1vE1sUMe")
project = rf.workspace("dummy-2qhux").project("pinewood_finals")
version = project.version(1)
dataset = version.download("yolov11")

# Train the YOLO model
model = YOLO("yolo11n-seg.pt")
model.train(
    data="/content/pinewood_finals-1/data.yaml",  # Path to dataset YAML
    epochs=60,  # Number of training epochs
    imgsz=1024,  # Training image size
    device=0  # Device to run on, e.g., GPU index
)

# Load the trained model
model = YOLO('/content/runs/segment/train/weights/best.pt')

# Set up paths for input and output
input_folder = '/content/drive/My Drive/IMAGES_PREDICT'
output_folder = '/content/drive/My Drive/IMAGES_RESULTS_MASKED_IMAGE_WITH_WATER'
os.makedirs(output_folder, exist_ok=True)

# Process and mask images
for image_name in filter(lambda x: x.lower().endswith(('.png', '.jpg', '.jpeg')), os.listdir(input_folder)):
    image_path = os.path.join(input_folder, image_name)
    img = cv2.imread(image_path)

    # Run the model on the image
    results = model(image_path)
    mask_data = results[0].masks.cpu().numpy()

    # Create a binary mask
    mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    for contour in results[0].masks.xy:
        cv2.drawContours(mask, [contour.astype(np.int32)], -1, (255), cv2.FILLED)

    # Apply the mask to the image
    mask_3d = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    isolated = cv2.bitwise_and(img, mask_3d)

    # Save the result
    output_path = os.path.join(output_folder, image_name)
    cv2.imwrite(output_path, isolated)
    print(f"Processed and saved: {output_path}")

**Object Segmentation Without Water**

In [ ]:
# Initialize Roboflow and download the dataset
rf = Roboflow(api_key="cY2YZlXNZGwP1vE1sUMe")
project = rf.workspace("dummy-2qhux").project("pinewood_final_without_water")
version = project.version(1)
dataset = version.download("yolov11")

# Train the YOLO model
model = YOLO("yolo11n-seg.pt")
model.train(
    data="/content/pinewood_final_without_water-1/data.yaml",
    epochs=60,
    imgsz=1024,
    device=0,
)

# Load the trained model
model = YOLO('/content/runs/segment/train/weights/best.pt')

# Set up paths for input and output
input_folder = '/content/drive/My Drive/IMAGES_PREDICT'
output_folder = '/content/drive/My Drive/IMAGES_RESULTS_MASKED_IMAGE_WITHOUT_WATER'
os.makedirs(output_folder, exist_ok=True)

# Process and mask images
for image_name in filter(lambda x: x.lower().endswith(('.png', '.jpg', '.jpeg')), os.listdir(input_folder)):
    image_path = os.path.join(input_folder, image_name)
    img = cv2.imread(image_path)

    # Run the model on the image
    results = model(image_path)
    mask_data = results[0].masks.cpu().numpy()

    # Create a binary mask
    mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    for contour in results[0].masks.xy:
        cv2.drawContours(mask, [contour.astype(np.int32)], -1, (255), cv2.FILLED)

    # Apply the mask to the image
    mask_3d = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    isolated = cv2.bitwise_and(img, mask_3d)

    # Save the result
    output_path = os.path.join(output_folder, image_name)
    cv2.imwrite(output_path, isolated)
    print(f"Processed and saved: {output_path}")
    print(f"Processed and saved: {output_path}")